## Data Loading

In [50]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime

os.environ['PROJ_LIB'] = '/opt/anaconda3/share/proj'

fp = os.path.join('/', 
             'Users', 
             'jpcohen', 
             'EDS-220', 
             'eds220-hwk4', 
             'data', 
             'California_Fire_Perimeters_(1950+)')

fire_perim = gpd.read_file(fp)